# Consdb for LSSTCam

- Creation date : 2025-04-18
- last update : 2025-04-18
- https://usdf-rsp-dev.slac.stanford.edu/consdb/
- Schemes for constdb : https://sdm-schemas.lsst.io/
- Documentation : https://consdb.lsst.io/index.html


LSSTCam/runs/nightlyValidation/{day_obs}/<lsst_distrib_tag>/DM-50157

In [ ]:
from lsst.summit.utils import ConsDbClient

In [ ]:
import numpy as np
from astropy.table import Table, join
from astropy.time import Time

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib widget

from lsst.meas.algorithms.installGaussianPsf import FwhmPerSigma

from tqdm.notebook import tqdm

## Configuration

In [ ]:
os.environ["no_proxy"] += ",.consdb"

In [ ]:
url="http://consdb-pq.consdb:8080/consdb"

In [ ]:
consdb=ConsDbClient(url)

https://sdm-schemas.lsst.io/cdb_lsstcomcam.html#exposure

In [ ]:
# Query both consDB tables
#exposure = consdb.query("SELECT * FROM cdb_lsstcam.exposure WHERE science_program = 'BLOCK-351'")
#visits = consdb.query("SELECT * FROM cdb_lsstcam.visit1 WHERE science_program = 'BLOCK-351'")
#visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1_quicklook")
#visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1")



exposure = consdb.query("SELECT * FROM cdb_lsstcam.exposure WHERE day_obs >= 20250415")
visits = consdb.query("SELECT * FROM cdb_lsstcam.visit1 WHERE day_obs >= 20250415")
#visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1_quicklook")
visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1")

# Join using astropy's join function on 'visit_id'
exposure_join = exposure.rename_column("exposure_id", "visit_id")
merged_exposure = join(exposure, visits, keys="visit_id", join_type="inner")  
merged_visits = join(visits, visits_ql, keys="visit_id", join_type="inner")  

# Display or use the merged table
#print(merged_visits)

## Explore the visit info in consdb

In [ ]:
print(visits.columns)

In [ ]:
df_visits = visits.to_pandas()

### Observation dates

In [ ]:
df_visits["day_obs"].unique()

### Filters

In [ ]:
print(df_visits["physical_filter"].unique(),df_visits["band"].unique())

### Science program and observation reason

In [ ]:
df_visits["science_program"].unique()

In [ ]:
df_visits['observation_reason'].unique()

### Time exposure

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.hist(df_visits["exp_time"].values,bins=50,facecolor="b");
plt.show()

## Region in sky

In [ ]:
dict_b_to_col = {"u": "b", "g":"g", "r": "r", "i": 'orange', 'z': "magenta",'y':'purple'}

In [ ]:
col = df_visits["band"].map(lambda b:  dict_b_to_col[b] ).values

In [ ]:
col

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
df_visits.plot.scatter(x='s_ra',y='s_dec',ax=ax,c=col)
ax.grid()
ax.set_aspect("equal")
plt.show()

## Time dependence

In [ ]:
time = Time(visits['exp_midpt'])

In [ ]:
print('tmin = ',time.min(),' | tmax = ',time.max())

### Airmass vs Time

In [ ]:

# Plot the time vs the specified column
plt.figure(figsize=(8, 4))
plt.plot(time.plot_date, visits['airmass'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('Airmass')
plt.title(f'Time vs Airmass')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['air_temp'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('air_temp')
plt.title(f'air_temp vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['pressure'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('pressure')
plt.title(f'pressure vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['humidity'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('humidity')
plt.title(f'humidity vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['wind_speed'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('wind_speed')
plt.title(f'wind_speed vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['wind_dir'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('wind_dir')
plt.title(f'wind_dir vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['dimm_seeing'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('dimm_seeing')
plt.title(f'dimm_seeing vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['focus_z'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('focus_z')
plt.title(f'focus_z vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()